In [ ]:
!pip install scikit-learn

In [25]:
import pandas as pd 
import json
import numpy as np
from sklearn.metrics import r2_score

In [75]:
teams = ["Broncos", "Roosters", "Wests Tigers", "Rabbitohs", "Storm", "Eels", "Raiders", "Knights", "Dragons", "Sea Eagles", "Panthers", "Sharks", "Bulldogs", "Dolphins", "Titans", "Cowboys", "Warriors"]
variables =["Year", "Win", "Defense", "Attack", "Margin", "Home", "Versus",  "Round"]
years =  [2015, 2016, 2017, 2018, 2022, 2023]

In [65]:
years_arr = {}
with open('nrl_data_multi_years.json', 'r') as file:
    data = json.load(file)
    data = data['NRL']
    for year in years:
        years_arr[year] = data[years.index(year)][str(year)]
    

In [84]:
df = pd.DataFrame(columns=[f"{team} {variable}" for team in teams for variable in variables])


In [85]:
all_store = []
for year in years:
    for round in range(0, 21):
        round_data = years_arr[year][round][str(round+1)]
        
        # Create an empty feature array 
        round_store = np.zeros([len(teams)*len(variables)], dtype=int)
        round_teams = []
        for game in round_data:
            h_team = game['Home']
            h_score = int(game['Home_Score'])
            a_team = game['Away']
            a_score = int(game['Away_Score'])
            
            # win or lose
            h_team_win = h_score >= a_score
            a_team_win = a_score >= h_score
            
            # home
            h_home = 1
            a_home = 0
            
            # versus
            h_versus = teams.index(a_team)
            a_versus = teams.index(h_team)
            
            # Defense (points let in)
            h_team_defense = a_score
            a_team_defense = h_score  
            
            # Attack Points scored
            h_team_attack = h_score 
            a_team_attack = a_score   
            
            # Margin
            h_team_margin =  h_score-a_score   
            a_team_margin =  a_score-h_score        
            
            # Keep track of which teams played to work out which teams had a bye 
            round_teams.append(h_team)
            round_teams.append(a_team)
            
            
            # Find the index of the team in the overaching array 
            a_team_idx = teams.index(a_team)
            h_team_idx = teams.index(h_team)
            
            # Feature map index
            a_team_idx_fm = a_team_idx*len(variables)
            h_team_idx_fm = h_team_idx*len(variables)
            
            # input(round_store)
            # input(a_team_idx_fm)
            # populate the data
            round_store[a_team_idx_fm] = year
            round_store[a_team_idx_fm+1] = a_team_win
            round_store[a_team_idx_fm+2] = a_team_defense
            round_store[a_team_idx_fm+3] = a_team_attack
            round_store[a_team_idx_fm+4] = a_team_margin
            round_store[a_team_idx_fm+5] = a_home
            round_store[a_team_idx_fm+6] = a_versus
            round_store[a_team_idx_fm+7] = round+1
            round_store[h_team_idx_fm] = year
            round_store[h_team_idx_fm+1] = h_team_win
            round_store[h_team_idx_fm+2] = h_team_defense
            round_store[h_team_idx_fm+3] = h_team_attack
            round_store[h_team_idx_fm+4] = h_team_margin
            round_store[h_team_idx_fm+5] = h_home
            round_store[h_team_idx_fm+6] = h_versus
            round_store[h_team_idx_fm+7] = round+1
            
        # input(round_teams)
        bye_teams = list(set(teams) - set(round_teams))
        for bye_team in bye_teams:
            b_team_idx = teams.index(bye_team)
            b_team_idx_fm = b_team_idx*len(variables)
            round_store[b_team_idx_fm] = -1
            round_store[b_team_idx_fm+1] = -1
            round_store[b_team_idx_fm+2] = -1
            round_store[b_team_idx_fm+3] = -1
            round_store[b_team_idx_fm+4] = 0
            round_store[b_team_idx_fm+5] = -1
            round_store[b_team_idx_fm+6] = -1
            round_store[b_team_idx_fm+7] = -1
            
        all_store.append(round_store)
        # Create an array with the same size as the DataFrame

        # Add the new row to the DataFrame using loc
        df.loc[len(df)] = round_store
df

,Broncos Year,Broncos Win,Broncos Defense,Broncos Attack,Broncos Margin,Broncos Home,Broncos Versus,Broncos Round,Roosters Year,Roosters Win,...,Cowboys Versus,Cowboys Round,Warriors Year,Warriors Win,Warriors Defense,Warriors Attack,Warriors Margin,Warriors Home,Warriors Versus,Warriors Round
0,2015,0,36,6,-30,1,3,1,2015,1,...,1,1,2015,0,24,14,-10,0,7,1
1,2015,1,2,10,8,0,11,2,2015,0,...,7,2,2015,1,6,18,12,0,6,2
2,2015,1,22,44,22,1,15,3,2015,1,...,0,3,2015,1,16,29,13,1,5,3
3,2015,1,16,24,8,0,16,4,2015,1,...,4,4,2015,0,24,16,-8,1,0,4
4,2015,1,16,26,10,0,14,5,2015,0,...,10,5,2015,0,30,14,-16,0,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,2023,0,18,12,-6,1,14,17,2023,0,...,3,17,2023,1,18,48,30,0,8,17
122,2023,1,16,24,8,1,13,18,2023,0,...,2,18,2023,0,28,6,-22,1,3,18
123,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,...,-1,-1,2023,1,10,46,36,0,5,19
124,2023,1,24,44,20,0,12,20,2023,0,...,9,20,2023,1,12,44,32,1,11,20


In [118]:
GAME_HISTORY = 5

def get_game_history(year, round, team):
    # Filter by year
    filtered_df = df[df[team + " Year"] == year]
    win = filtered_df[team+ " Win"][round-GAME_HISTORY:round].mean()
    defense = filtered_df[team+ " Defense"][round-GAME_HISTORY:round].mean()
    attack = filtered_df[team+ " Attack"][round-GAME_HISTORY:round].mean()
    margin = filtered_df[team+ " Margin"][round-GAME_HISTORY:round].mean()
    return win, defense, attack, margin
    

In [94]:
input = []
output = [] 
# Input: Team, Other Team Team Stats, Other Team Stats
# Output: Team, win/lose, other team, win/lose


for team in teams:
    # Skip the game history as we dont want to break the calibration
    versed_teams = df[team+ " Versus"]
    wins = df[team+ " Win"]
    rounds = df[team+ " Round"]
    years = df[team+ " Year"]
    
    
    c_team_idx = teams.index(team)
    
    
    
    for versed_team, win, round, year in zip(versed_teams, wins, rounds, years):
        # Skip all byes  or games with no momentum
        if win == -1 or round < GAME_HISTORY:
            continue
        
        
        c_win, c_defense, c_attack, c_margin = get_game_history(year, round, team)
        v_win, v_defense, v_attack, v_margin = get_game_history(year, round, teams[versed_team])
        
        
        
        # dosent take into account for ties, look into this harder later 
        if win == 1:
            v_win_ = 0
        else:
            v_win_ = 1
        
        # Current team, vsed team, who wins, current_team_stats, vsed_team_stats
        input.append([c_team_idx, versed_team, c_win, c_defense, c_attack, c_margin, v_win, v_defense, v_attack, v_margin])
        output.append([c_team_idx, versed_team, win, v_win_])


In [96]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

# Create a Random Forest Regressor model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Perform k-fold cross-validation (let's say k=5)
k_folds = 5
mse_scores = cross_val_score(rf_model, X_train, y_train, scoring='neg_mean_squared_error', cv=k_folds)

# Convert negative MSE scores to positive (mean_squared_error uses negative values for scoring)
mse_scores = -mse_scores

# Calculate Root Mean Squared Error (RMSE) and R-squared (R2) score from the MSE scores
rmse_scores = np.sqrt(mse_scores)
r2_scores = cross_val_score(rf_model, X_train, y_train, scoring='r2', cv=k_folds)

# Print the results
print("Cross-Validation RMSE scores:", rmse_scores)
print("Cross-Validation R2 scores:", r2_scores)

# Fit the model on the whole training data
rf_model.fit(X_train, y_train)

# Evaluate the model on the test set
y_pred = rf_model.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)
test_rmse = np.sqrt(test_mse)
test_r2 = r2_score(y_test, y_pred)

print("Test RMSE:", test_rmse)
print("Test R2 score:", test_r2)


Cross-Validation RMSE scores: [0.57168504 0.63162972 0.58871087 0.6927059  0.72166518]
Cross-Validation R2 scores: [0.64269475 0.66659882 0.70983583 0.56179759 0.41714925]
Test RMSE: 0.5255137542275682
Test R2 score: 0.7541925208681841


In [97]:
# Make predictions on the test set
predictions = model.predict(X_test)

# Evaluate the model using Mean Squared Error (MSE)
# Smaller MSE == performance imporvements
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)

# Root mean square error is the average error in the predictions
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

# 1 is a perfect fit with r2 score
r2 = r2_score(y_test, predictions)
print("R-squared score:", r2)


Mean Squared Error: 0.10746646807005202
Root Mean Squared Error: 0.3278207865130764
R-squared score: 0.7413482149265433


In [124]:
# teams = ["Broncos", "Roosters", "Wests Tigers", "Rabbitohs", "Storm", "Eels", "Raiders", "Knights", "Dragons", "Sea Eagles", "Panthers", "Sharks", "Bulldogs", "Dolphins", "Titans", "Cowboys", "Warriors"]


wkd_matches = [["Broncos", "Roosters"], ["Wests Tigers", "Rabbitohs"], ["Storm", "Eels"], ["Raiders", "Knights"], ["Dragons", "Sea Eagles"], ["Panthers", "Sharks"], ["Bulldogs", "Dolphins"], ["Titans", "Cowboys"]]

# wkd_matches = [["Dragons", "Wests Tigers"], ["Warriors", "Raiders"], ["Rabbitohs", "Broncos"], ["Titans", "Roosters"], ["Knights", "Storm"], ["Cowboys", "Eels"], ["Panthers", "Bulldogs"], ["Sharks", "Sea Eagles"]]

for wkd_match in wkd_matches:

    team_1 = int(teams.index(wkd_match[0]))
    team_2 = int(teams.index(wkd_match[1]))
    # input(team_1)

    c_win, c_defense, c_attack, c_margin = get_game_history(2023, 22, teams[int(team_1)])
    v_win, v_defense, v_attack, v_margin =  get_game_history(2023, 22, teams[int(team_2)])
    

    predictions = model.predict([[team_1, team_2, c_win, c_defense, c_attack, c_margin, v_win, v_defense, v_attack, v_margin]])
    predictions = predictions[0]
    if predictions[2] > predictions[3]:
        print(f"{teams[team_1]} wins\t\t {teams[team_1]}: {predictions[2]:.4f}\t{teams[team_2]}: {predictions[3]:.4f}")
    else:
        print(f"{teams[team_2]} wins\t\t {teams[team_1]}: {predictions[2]:.4f}\t{teams[team_2]}: {predictions[3]:.4f}")


Roosters wins		 Broncos: 0.4320	Roosters: 0.5414
Rabbitohs wins		 Wests Tigers: 0.0949	Rabbitohs: 0.4854
Eels wins		 Storm: -0.0032	Eels: 0.8511
Knights wins		 Raiders: -0.1680	Knights: 1.0625
Dragons wins		 Dragons: 0.9646	Sea Eagles: 0.2486
Panthers wins		 Panthers: 1.3888	Sharks: -0.2167
Bulldogs wins		 Bulldogs: 0.6584	Dolphins: 0.6479
Cowboys wins		 Titans: 0.2469	Cowboys: 1.1213
